In [1]:
import sys
sys.executable


sys.path

['C:\\Users\\Joelr\\Anaconda3\\envs\\tf_EPQ\\python36.zip',
 'C:\\Users\\Joelr\\Anaconda3\\envs\\tf_EPQ\\DLLs',
 'C:\\Users\\Joelr\\Anaconda3\\envs\\tf_EPQ\\lib',
 'C:\\Users\\Joelr\\Anaconda3\\envs\\tf_EPQ',
 '',
 'C:\\Users\\Joelr\\AppData\\Roaming\\Python\\Python36\\site-packages',
 'C:\\Users\\Joelr\\Anaconda3\\envs\\tf_EPQ\\lib\\site-packages',
 'C:\\Users\\Joelr\\Anaconda3\\envs\\tf_EPQ\\lib\\site-packages\\win32',
 'C:\\Users\\Joelr\\Anaconda3\\envs\\tf_EPQ\\lib\\site-packages\\win32\\lib',
 'C:\\Users\\Joelr\\Anaconda3\\envs\\tf_EPQ\\lib\\site-packages\\Pythonwin',
 'C:\\Users\\Joelr\\Anaconda3\\envs\\tf_EPQ\\lib\\site-packages\\IPython\\extensions',
 'C:\\Users\\Joelr\\.ipython']

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input
#from keras.utils.vis_utils import plot_model
from tensorflow.keras import layers, preprocessing, callbacks
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.models import Sequential
import matplotlib.pyplot as plt
from IPython.display import Image
import matplotlib.cm as cm
from tensorflow.keras.applications import ResNet50, VGG16
import datetime, os
import numpy as np
import seaborn as sn
import pandas as pd
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.utils import class_weight

%load_ext tensorboard

In [2]:
# Important variables
EPOCHS = 50

TRAIN_DIR = "C:/Users\Joelr/Keras_EPQ/x_rays/train"
TEST_DIR = "C:/Users\Joelr/Keras_EPQ/x_rays/test"
PRED_DIR = "C:/Users/Joelr/Keras_EPQ/x_rays/prediction"

BATCH_SIZE = 16

IMG_HEIGHT = 150
IMG_WIDTH = 150

CLASSES = ["0","1","2","3","4"]


In [3]:
# Data preprocessing

train_gen = tf.keras.preprocessing.image.ImageDataGenerator(
    rotation_range=5,
    width_shift_range=2.0,
    height_shift_range=2.0,
    zoom_range=0.125,
    channel_shift_range=0.0, #was 3.0
    fill_mode="nearest",
    horizontal_flip=True,
    vertical_flip=True,
    rescale=1./255
)

test_gen = keras.preprocessing.image.ImageDataGenerator(rescale=1./255)



train_data_gen = train_gen.flow_from_directory(directory= TRAIN_DIR,
                                                     batch_size=BATCH_SIZE,
                                                     shuffle=True,
                                                     target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                     classes = CLASSES)

test_data_gen = test_gen.flow_from_directory(directory= TEST_DIR,
                                                     batch_size=BATCH_SIZE,
                                                     shuffle=False,
                                                     target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                     classes = CLASSES)


pred_data_gen = test_gen.flow_from_directory(directory= PRED_DIR,
                                                     batch_size= 18 ,
                                                     shuffle=False,
                                                     target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                     classes = "0")


 
# Alter class weights to to counter imbalance

weight = class_weight.compute_class_weight(
               'balanced',
                np.unique(train_data_gen.classes), 
                train_data_gen.classes)

CLASS_WEIGHTS = dict(enumerate(weight))

Found 7392 images belonging to 5 classes.
Found 462 images belonging to 5 classes.
Found 18 images belonging to 1 classes.


c:\users\joelr\anaconda3\envs\gamer\lib\site-packages\sklearn\utils\validation.py:70: FutureWarning: Pass classes=[0 1 2 3 4], y=[0 0 0 ... 4 4 4] as keyword args. From version 0.25 passing these as positional arguments will result in an error
  FutureWarning)


In [4]:
# Define model

base = VGG16(weights='imagenet', include_top=False,input_tensor=Input(shape=(IMG_HEIGHT, IMG_WIDTH, 3)))

# Take output from pre-trained and put through custom classifier
x = base.output
x = tf.keras.layers.GlobalMaxPooling2D()(x)
x =tf.keras.layers.Dense(1024, activation='relu')(x)
x =tf.keras.layers.Dense(512, activation='relu')(x)
x =tf.keras.layers.Dense(64, activation='relu')(x)
output = tf.keras.layers.Dense(5, activation='softmax')(x) #OUtput is an array of probabilities 0-1

model = Model(inputs=base.input, outputs=output)

In [5]:
for i, layer in enumerate(model.layers):
    print(i, layer.name)

model.summary()

#Select which layers to train
for layer in model.layers[:10]:
   layer.trainable = False
for layer in model.layers[10:]:
   layer.trainable = True


from tensorflow.keras.optimizers import SGD, Adam, RMSprop
#model.compile(optimizer=SGD(learning_rate=0.001), loss='categorical_crossentropy', metrics=['acc'])
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['acc'])

0 input_1
1 block1_conv1
2 block1_conv2
3 block1_pool
4 block2_conv1
5 block2_conv2
6 block2_pool
7 block3_conv1
8 block3_conv2
9 block3_conv3
10 block3_pool
11 block4_conv1
12 block4_conv2
13 block4_conv3
14 block4_pool
15 block5_conv1
16 block5_conv2
17 block5_conv3
18 block5_pool
19 global_max_pooling2d
20 dense
21 dense_1
22 dense_2
23 dense_3
Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 150, 150, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 150, 150, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 150, 150, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 75, 75, 64)        0         
__________________________________

In [7]:
model.save("testVGG16.h5",include_optimizer=True)

In [7]:
#Train

logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1,  write_graph=True)


reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=3,
                              min_lr=0,
                              cooldown=0,
                              min_delta=0.001,
                              verbose=1)

early_stop = tf.keras.callbacks.EarlyStopping(
    monitor="val_loss",
    min_delta=0.1,
    patience=15,
    verbose=0,
    mode="auto",
    baseline=None,
    restore_best_weights=True,
)

callbacks = [tensorboard_callback, reduce_lr]

model.fit(x = train_data_gen,
         epochs = EPOCHS,
         validation_data = test_data_gen,
          callbacks=callbacks,
          class_weight = CLASS_WEIGHTS
         )


Epoch 1/50
  1/462 [..............................] - ETA: 0s - loss: 2.0037 - acc: 0.1250WARNING:tensorflow:From c:\users\joelr\anaconda3\envs\gamer\lib\site-packages\tensorflow\python\ops\summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
462/462 [==============================] - 99s 215ms/step - loss: 1.6280 - acc: 0.1292 - val_loss: 1.6096 - val_acc: 0.1342
Epoch 2/50
462/462 [==============================] - 96s 208ms/step - loss: 1.6097 - acc: 0.1909 - val_loss: 1.6096 - val_acc: 0.1342
Epoch 3/50
462/462 [==============================] - 96s 208ms/step - loss: 1.6098 - acc: 0.1350 - val_loss: 1.6093 - val_acc: 0.2165
Epoch 4/50
462/462 [==============================] - ETA: 0s - loss: 1.6097 - acc: 0.2545
Epoch 00004: ReduceLROnPlateau reducing learning rate to 0.00020000000949949026.
462/462 [==============================] - 96s 209m

KeyboardInterrupt: 

In [ ]:
model = load_model("VGG16.h5")

In [ ]:
preds_array = model.predict(test_data_gen)
preds = np.argmax(preds_array, axis=1)

conf_mat = confusion_matrix(test_data_gen.classes, preds)
print('Confusion Matrix')
print(conf_mat,"\n")
print('Classification Report')
print(classification_report(test_data_gen.classes, preds, target_names=CLASSES))

# Precision = True Positives / True Positives + False Positives i.e. Out of all the knees labelled as 4, how many were actually 4
# Recall = True Pos / True pos + False neg i.e. Out of all the knees that were actually 4, how many were labelled 4

In [ ]:
#Heatmap of confusion matrix

%matplotlib
sn.heatmap(conf_mat, annot=True, 
            fmt='', cmap='Blues')


plt.ylabel('True')
plt.xlabel('Predicted')
b, t = plt.ylim()
b += 0.5 
t -= 0.5 
plt.ylim(b, t)
plt.show() 

In [ ]:
#Gives % confidence predictions and shows on bar chart.
x, y = test_data_gen.next()
pred_array = model.predict(x)
y = np.argmax(y, axis=1)

for i in range(BATCH_SIZE):
    fig, (ax1, ax2) = plt.subplots(1, 2)
    
    ax1.imshow(x[i])
    label = "True Class " + str(y[i] )
    ax1.set_xlabel(label)

    ax2.bar(CLASSES, pred_array[i]*100, width=0.8, bottom=None, align='center', data=None)
    ax2.set_ylim([0,100])
    ax2.set_ylabel('Prediction Confidence(%)')
    ax2.set_xlabel("Classes")
    
    fig.tight_layout()
    fig.show()

In [ ]:
#Grad CAM from https://keras.io/examples/vision/grad_cam/#the-gradcam-algorithm

img_size = (150, 150)

last_conv_layer_name = "block5_conv3"

classifier_layer_names = [
    "global_max_pooling2d",
    "dense",
    "dense_1",
    "dense_2",
    "dense_3"
]

img_path = "C://Users//Joelr//Keras_EPQ//x_rays//test//2//9024940R.png"


display(Image(img_path))

In [ ]:
#Grad CAM from https://keras.io/examples/vision/grad_cam/#the-gradcam-algorithm


def get_img_array(img_path, size):
    # `img` is a PIL image of size 299x299
    img = keras.preprocessing.image.load_img(img_path, target_size=size)
    # `array` is a float32 Numpy array of shape (299, 299, 3)
    array = keras.preprocessing.image.img_to_array(img)
    # We add a dimension to transform our array into a "batch"
    # of size (1, 299, 299, 3)
    array = np.expand_dims(array, axis=0)
    return array

def make_gradcam_heatmap(img_array, model, last_conv_layer_name, classifier_layer_names):
    # First, we create a model that maps the input image to the activations
    # of the last conv layer
    last_conv_layer = model.get_layer(last_conv_layer_name)
    last_conv_layer_model = keras.Model(model.inputs, last_conv_layer.output)

    # Second, we create a model that maps the activations of the last conv
    # layer to the final class predictions
    classifier_input = keras.Input(shape=last_conv_layer.output.shape[1:])
    x = classifier_input
    for layer_name in classifier_layer_names:
        x = model.get_layer(layer_name)(x)
    classifier_model = keras.Model(classifier_input, x)

    # Then, we compute the gradient of the top predicted class for our input image
    # with respect to the activations of the last conv layer
    with tf.GradientTape() as tape:
        # Compute activations of the last conv layer and make the tape watch it
        last_conv_layer_output = last_conv_layer_model(img_array)
        tape.watch(last_conv_layer_output)
        # Compute class predictions
        preds = classifier_model(last_conv_layer_output)
        top_pred_index = tf.argmax(preds[0])
        top_class_channel = preds[:, top_pred_index]

    # This is the gradient of the top predicted class with regard to
    # the output feature map of the last conv layer
    grads = tape.gradient(top_class_channel, last_conv_layer_output)

    # This is a vector where each entry is the mean intensity of the gradient
    # over a specific feature map channel
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

    # We multiply each channel in the feature map array
    # by "how important this channel is" with regard to the top predicted class
    last_conv_layer_output = last_conv_layer_output.numpy()[0]
    pooled_grads = pooled_grads.numpy()
    for i in range(pooled_grads.shape[-1]):
        last_conv_layer_output[:, :, i] *= pooled_grads[i]

    # The channel-wise mean of the resulting feature map
    # is our heatmap of class activation
    heatmap = np.mean(last_conv_layer_output, axis=-1)

    # For visualization purpose, we will also normalize the heatmap between 0 & 1
    heatmap = np.maximum(heatmap, 0) / np.max(heatmap)
    return heatmap

In [ ]:
#Grad CAM from https://keras.io/examples/vision/grad_cam/#the-gradcam-algorithm


img_array = get_img_array(img_path, size=img_size)

# Make model

# Print what the top predicted class is
preds = model.predict(img_array)
print("Predicted:",np.argmax(preds[0]))

# Generate class activation heatmap
heatmap = make_gradcam_heatmap(
    img_array, model, last_conv_layer_name, classifier_layer_names
)

# Display heatmap
plt.matshow(heatmap)
plt.show()

In [ ]:
#Grad CAM from https://keras.io/examples/vision/grad_cam/#the-gradcam-algorithm

def CAM(img_path):
    img = keras.preprocessing.image.load_img(img_path)
    img = keras.preprocessing.image.img_to_array(img)
    
    img_array = get_img_array(img_path, size=img_size)
    
    preds = model.predict(img_array)
    print("Predicted:",np.argmax(preds[0]))
    
    heatmap = make_gradcam_heatmap(img_array, model, last_conv_layer_name, classifier_layer_names)

    # We rescale heatmap to a range 0-255
    heatmap = np.uint8(255 * heatmap)

    # We use jet colormap to colorize heatmap
    jet = cm.get_cmap("jet")

    # We use RGB values of the colormap
    jet_colors = jet(np.arange(256))[:, :3]
    jet_heatmap = jet_colors[heatmap]

    # We create an image with RGB colorized heatmap
    jet_heatmap = keras.preprocessing.image.array_to_img(jet_heatmap)
    jet_heatmap = jet_heatmap.resize((img.shape[1], img.shape[0]))
    jet_heatmap = keras.preprocessing.image.img_to_array(jet_heatmap)

    # Superimpose the heatmap on original image
    superimposed_img = jet_heatmap * 0.4 + img
    superimposed_img = keras.preprocessing.image.array_to_img(superimposed_img)

    # Save the superimposed image
    save_path = "C:/Users/Joelr/Keras_EPQ/CAM/vgg16/" + img_path[37] + "/" + img_path[39:]
    superimposed_img.save(save_path)

    # Display Grad CAM
    display(Image(save_path))

In [ ]:
#Dont use this a lot

from os import listdir
import fnmatch

folders = os.listdir("C:/Users/Joelr/Keras_EPQ/x_rays/test")


import glob
files = glob.glob("C:/Users/Joelr/Keras_EPQ/x_rays/test/0/*.png")

# 37 index for class  39 index onwards for name

for file in files:
    CAM(file)
    #print(files[file])
